In [280]:
import os
import pandas as pd

In [319]:
os.chdir('/Users/manikant/Desktop/GT_files/GA/Iowa')

In [350]:
popdf=pd.read_csv('Cleaned_Block_Group_Pop_Centers.csv')
# racedf=pd.read_csv('RAWomen_BlockGroup_For_Iowa.csv')
bg_long=pd.read_csv('BG_hosp_long_new.csv')

racedf=pd.read_csv('Race_df.csv')
povertydf=pd.read_csv('Poverty_df.csv')


In [336]:
pattern = r'(?P<Part1>\d+)(?P<Part2>[A-Za-z]+)(?P<Part3>\d{2})(?P<Part4>\d{3})(?P<Part5>\d{6})(?P<Part6>\d)'

# Extracting parts into separate columns
df = bg_long['Geography'].str.extract(pattern)

# Renaming the columns for clarity
df.columns = ['Part1', 'Part2', 'State', 'County', 'Tract', 'BlockGroup']

df['Geography']=bg_long['Geography']
df=df.drop_duplicates().reset_index(drop=True)
df=df.merge(bg_long[['Geography','County']],how='left',on='Geography')
df=df.rename(columns={'County_x':'CountFP','County_y':'County'})

In [337]:
df.head()

,Part1,Part2,State,CountFP,Tract,BlockGroup,Geography,County
0,1500000,US,19,001,960100,1,1500000US190019601001,Adair
1,1500000,US,19,001,960100,1,1500000US190019601001,Adair
2,1500000,US,19,001,960100,1,1500000US190019601001,Adair
3,1500000,US,19,001,960100,1,1500000US190019601001,Adair
4,1500000,US,19,001,960100,1,1500000US190019601001,Adair


In [338]:
split_values = racedf['Geographic Area Names'].str.split(';', expand=True)

# Concatenating the new DataFrame with the original one
racedf = pd.concat([racedf, split_values], axis=1)
racedf[2]=racedf[2].str.replace(' County','').str.lstrip()
racedf[0]=racedf[0].str.replace('Block Group','').str.strip()
racedf[1]=racedf[1].str.replace('Census Tract','').str.strip()

racedf.rename(columns={2:'County',0:'BGFP',1:'TractFP'},inplace=True)

racedf=racedf.merge(df[['County','CountFP']].drop_duplicates(),how='left',on='County')
racedf['TractFP']=pd.to_numeric(racedf['TractFP'])

racedf['TractFP']=racedf['TractFP']*100


racedf['TractFP']=racedf['TractFP'].astype(int)
racedf['TractFP'] =racedf['TractFP'].astype(str).apply(lambda x: x.zfill(6))
racedf['TractFP']=racedf['TractFP'].astype(str)



racedf['Geography']=racedf.apply(lambda row: '1500000US19'+row['CountFP']+row['TractFP']+row['BGFP'],axis=1)


In [339]:
racedf.head()

,Geographic Area Names,Total Female Age 18-44 years,"Total Native Hawaiian & Other Pacific Islander Female, 18-44","Total Asian Female, 18-44","Total American Indian and Alaska Native Female, 18-44","Total Black or African American Female, 18-44","Total White Female, 18-44",BGFP,TractFP,County,3,CountFP,Geography
0,Block Group 1; Census Tract 9602; Adair County...,45,0.0,0.0,0.0,0.0,45.0,1,960200,Adair,Iowa,001,1500000US190019602001
1,Block Group 2; Census Tract 9602; Adair County...,335,0.0,0.0,0.0,0.0,335.0,2,960200,Adair,Iowa,001,1500000US190019602002
2,Block Group 1; Census Tract 9603; Adair County...,144,0.0,0.0,0.0,0.0,144.0,1,960300,Adair,Iowa,001,1500000US190019603001
3,Block Group 2; Census Tract 9603; Adair County...,42,0.0,0.0,0.0,0.0,42.0,2,960300,Adair,Iowa,001,1500000US190019603002
4,Block Group 3; Census Tract 9603; Adair County...,97,0.0,0.0,0.0,0.0,97.0,3,960300,Adair,Iowa,001,1500000US190019603003


In [340]:
racedf.rename(columns={'Total Native Hawaiian & Other Pacific Islander Female, 18-44':'Other',
                      'Total Asian Female, 18-44':'Asian',
                      'Total American Indian and Alaska Native Female, 18-44':'Native',
                      'Total Black or African American Female, 18-44':'Black',
                      'Total White Female, 18-44':'White'},inplace=True)

racedf.drop(columns=['BGFP','TractFP','County',3,'CountFP'])

racedf=racedf[['Geographic Area Names','Geography','Asian','Black','Native','White','Other']]


In [341]:
racedf.head()

,Geographic Area Names,Geography,Asian,Black,Native,White,Other
0,Block Group 1; Census Tract 9602; Adair County...,1500000US190019602001,0.0,0.0,0.0,45.0,0.0
1,Block Group 2; Census Tract 9602; Adair County...,1500000US190019602002,0.0,0.0,0.0,335.0,0.0
2,Block Group 1; Census Tract 9603; Adair County...,1500000US190019603001,0.0,0.0,0.0,144.0,0.0
3,Block Group 2; Census Tract 9603; Adair County...,1500000US190019603002,0.0,0.0,0.0,42.0,0.0
4,Block Group 3; Census Tract 9603; Adair County...,1500000US190019603003,0.0,0.0,0.0,97.0,0.0


In [342]:
racedf.to_csv('race_df.csv',index=False)

In [314]:
povertydf.head()

,Geographic Area Names,Total Female Age 18-44 years,Geography,Proportion Below Poverty,Proportion Above Poverty
0,Block Group 1; Census Tract 9601; Adair County...,321,1400000US19001960100,47.0,274.0
1,Block Group 2; Census Tract 9601; Adair County...,186,1400000US19001960100,27.0,159.0
2,Block Group 3; Census Tract 9601; Adair County...,188,1400000US19001960100,27.0,161.0
3,Block Group 1; Census Tract 9602; Adair County...,45,1400000US19001960200,5.0,40.0
4,Block Group 2; Census Tract 9602; Adair County...,335,1400000US19001960200,38.0,297.0


In [343]:
povertydf=povertydf.merge(racedf[['Geographic Area Names','Geography']],how='left',on='Geographic Area Names')


In [345]:
povertydf.head()

,Geographic Area Names,Total Female Age 18-44 years,Geography_x,Proportion Below Poverty,Proportion Above Poverty,Geography_y
0,Block Group 1; Census Tract 9601; Adair County...,321,1400000US19001960100,47.0,274.0,NaN
1,Block Group 2; Census Tract 9601; Adair County...,186,1400000US19001960100,27.0,159.0,NaN
2,Block Group 3; Census Tract 9601; Adair County...,188,1400000US19001960100,27.0,161.0,NaN
3,Block Group 1; Census Tract 9602; Adair County...,45,1400000US19001960200,5.0,40.0,1500000US190019602001
4,Block Group 2; Census Tract 9602; Adair County...,335,1400000US19001960200,38.0,297.0,1500000US190019602002


In [348]:
povertydf.to_csv('poverty_df.csv')

In [351]:
racedf.head()

,Geographic Area Names,Geography,Asian,Black,Native,White,Other
0,Block Group 1; Census Tract 9602; Adair County...,1500000US190019602001,0.0,0.0,0.0,45.0,0.0
1,Block Group 2; Census Tract 9602; Adair County...,1500000US190019602002,0.0,0.0,0.0,335.0,0.0
2,Block Group 1; Census Tract 9603; Adair County...,1500000US190019603001,0.0,0.0,0.0,144.0,0.0
3,Block Group 2; Census Tract 9603; Adair County...,1500000US190019603002,0.0,0.0,0.0,42.0,0.0
4,Block Group 3; Census Tract 9603; Adair County...,1500000US190019603003,0.0,0.0,0.0,97.0,0.0


In [352]:
povertydf.head()

,Geographic Area Names,Below_Poverty,Above_Poverty,Geography
0,Block Group 1; Census Tract 9601; Adair County...,47,274,1500000US190019601001
1,Block Group 2; Census Tract 9601; Adair County...,27,159,1500000US190019601002
2,Block Group 3; Census Tract 9601; Adair County...,27,161,1500000US190019601003
3,Block Group 1; Census Tract 9602; Adair County...,5,40,1500000US190019602001
4,Block Group 2; Census Tract 9602; Adair County...,38,297,1500000US190019602002
